Add your name and PID
<br> Kush Parmar A14032759<br> Alex Biz A14455796 <br> Jessica Cramer A12606890

## Introduction

### Overview

### Research Question

### Hypothesis

### Background

###  Dataset Description

## Data Cleaning and Pre-processing

### Setup

In [13]:
# import libraries
import pandas as pd
import spotipy
from profanity_check import predict, predict_prob
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials

In [14]:
# load in raw data
raw_data = pd.read_csv("./dataset/with_prof_no_norm.csv")
# raw_data.index = raw_data["index"]
# raw_data = raw_data.drop(columns=["index"])

# add popularity column
# raw_data["pop"] = np.zeros(len(raw_data))

raw_data

,Unnamed: 0,index,song,year,artist,genre,lyrics,pop,prof
0,0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\r\r\r\r\r\nYou know I'...",42.0,2
1,5,5,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard\r\r\r\r\r\nChurch bells ringing\r\r\r\...,46.0,0
2,6,6,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend\r\...,33.0,0
3,16,16,poison,2009,beyonce-knowles,Pop,You're bad for me I clearly get it\r\r\r\r\r\n...,39.0,2
4,18,18,beautiful-liar,2007,beyonce-knowles,Pop,Ay\r\r\r\r\r\nAy\r\r\r\r\r\nAy (Nobody likes t...,61.0,0
5,19,19,beautiful-liar-spanish,2007,beyonce-knowles,Pop,Ay! Ay!\r\r\r\r\r\n(Nobody likes being played)...,34.0,0
6,20,20,beautiful-liar-spanglish-version,2007,beyonce-knowles,Pop,Ay! Ay!\r\r\r\r\r\n(Nobody likes being played)...,33.0,0
7,24,24,bello-embustero,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\r\r\r\r\r\nOh! Beyonc, Beyonc\r\r\...",34.0,0
8,29,29,if,2007,beyonce-knowles,Pop,He is always laughin' and flirting with me\r\r...,68.0,0
9,31,31,bello-embustero-beautiful-liar-spanish-version,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\r\r\r\r\r\nOh! Beyonce, Beyonce\r\...",34.0,0


In [3]:
# get popularity data from spotify API

# Setup spotify API 
# client_manage = SpotifyClientCredentials(client_id="a47de82ee2f24629ba951b0d1beea37c" , client_secret="539e6aca7ae648b981d8dc494f276cd8")
# spot = spotipy.Spotify(client_credentials_manager=client_manage)

In [4]:
# iterate over all songs 
# for i in range(359999, len(raw_data)):
    # song = raw_data.iloc[i]["song"]
    # artist = raw_data.iloc[i]["artist"]
    
    # replace hyphen with spaces
    # song = song.replace("-", " ")
    # artist = artist.replace("-", " ")
    
    # get popularity ratings - maybe other stuff later on
    # result = spot.search("track: " + song + " artist: " + artist, limit=3)
    # result = result['tracks']['items']
    
    # go through list, and find song
    # if len(result) != 0:
        # name = result[0]["name"]
        # pop = result[0]["popularity"]
        # raw_data.at[i,"pop"] = pop

    # print
    # print("iteration " + str(i) +"/" + str(len(raw_data)), end="\r")
    
    # save progress for every song, and keep track of progress
    # if (i % 1000) == 0:
        # write to file
        # raw_data.to_csv("./dataset/with_pop.csv")
        
# write to file
# raw_data.to_csv("./dataset/with_pop.csv")

### Cleaning

In [30]:
# Drop NaN lyrics
# raw_data.dropna(inplace=True, subset=["lyrics"])

# Drop 0 popularity
# raw_data = raw_data.drop(raw_data[raw_data["pop"] == 0].index)

# drop index columns from original data
# raw_data = raw_data.drop(columns=["index", "Unnamed: 0"])

# clean \rs from all lyrics
def clean_lyrics(lyrics):
    lyrics = lyrics.replace("\r", "")
    return lyrics
# raw_data["lyrics"] = raw_data["lyrics"].apply(clean_lyrics)
raw_data

,song,year,artist,genre,lyrics,pop,prof
0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu...",42.0,2
1,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard\nChurch bells ringing\nI heard\nA choi...,46.0,0
2,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend\nW...,33.0,0
3,poison,2009,beyonce-knowles,Pop,You're bad for me I clearly get it\nI don't se...,39.0,2
4,beautiful-liar,2007,beyonce-knowles,Pop,"Ay\nAy\nAy (Nobody likes to be played)\nOh, Be...",61.0,0
5,beautiful-liar-spanish,2007,beyonce-knowles,Pop,Ay! Ay!\n(Nobody likes being played)\nAy!\nOh!...,34.0,0
6,beautiful-liar-spanglish-version,2007,beyonce-knowles,Pop,Ay! Ay!\n(Nobody likes being played)\nAy!\nOh!...,33.0,0
7,bello-embustero,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\nOh! Beyonc, Beyonc\nOh! Beyonc, B...",34.0,0
8,if,2007,beyonce-knowles,Pop,He is always laughin' and flirting with me\nAn...,68.0,0
9,bello-embustero-beautiful-liar-spanish-version,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\nOh! Beyonce, Beyonce\nOh! Beyonce...",34.0,0


In [22]:
# add profanity column

def prof(lyrics):
    lyrics = lyrics.split("\n")
    prof = 0
    prof_arr = np.array(predict(lyrics))
    prof = prof_arr.sum()
    print(prof, end="\r")
    return prof

raw_data["prof"] = raw_data["lyrics"].apply(prof)

In [31]:
# raw_data.to_csv("./dataset/with_prof_no_norm.csv")

In [36]:
# add normalised column
# for i in range(len(raw_data)):
    # prof = raw_data.iloc[i]["prof"]
    # lines = len(raw_data.iloc[i]["lyrics"].split("\n"))
    # raw_data.at[i, "norm"] = prof / lines
    # print(i, end="\r")


In [37]:
raw_data

,song,year,artist,genre,lyrics,pop,prof,norm
0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu...",42.0,2,0.040000
1,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard\nChurch bells ringing\nI heard\nA choi...,46.0,0,0.000000
2,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend\nW...,33.0,0,0.000000
3,poison,2009,beyonce-knowles,Pop,You're bad for me I clearly get it\nI don't se...,39.0,2,0.023810
4,beautiful-liar,2007,beyonce-knowles,Pop,"Ay\nAy\nAy (Nobody likes to be played)\nOh, Be...",61.0,0,0.000000
5,beautiful-liar-spanish,2007,beyonce-knowles,Pop,Ay! Ay!\n(Nobody likes being played)\nAy!\nOh!...,34.0,0,0.000000
6,beautiful-liar-spanglish-version,2007,beyonce-knowles,Pop,Ay! Ay!\n(Nobody likes being played)\nAy!\nOh!...,33.0,0,0.000000
7,bello-embustero,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\nOh! Beyonc, Beyonc\nOh! Beyonc, B...",34.0,0,0.000000
8,if,2007,beyonce-knowles,Pop,He is always laughin' and flirting with me\nAn...,68.0,0,0.000000
9,bello-embustero-beautiful-liar-spanish-version,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\nOh! Beyonce, Beyonce\nOh! Beyonce...",34.0,0,0.000000


In [38]:
# raw_data.to_csv("./dataset/with_norm.csv")

## Data Analysis

In [1]:
# summary, and description of data

In [39]:
# EDA

In [40]:
# 1. prof, and pop  2. pop, and genre, and year 3. year and prof

## Privacy/Ethics Considerations

## Conclusion

## Discussion